### features:
* selected spectogram coef
* zero crossing rate
* selected chromogram values
* selected tempogram values
* wavelet of level 0

In [1]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, recall_score
import matplotlib.pyplot as plt
import itertools

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
csv_path = "/Users/ecem/Desktop/csv of scgs"
wavelet_csv = "/Users/ecem/Desktop/wavelet-csv"

## Load data

Here, our target is to predict disease of patients with multiple diseases. So; our y will be disease column of two column indicatin geach patients disease.

In [3]:
spec = pd.read_csv(csv_path + '/p-value-selected-intersection-multiple-dis-spec.csv', index_col = 0)
#spec  #uncomment to see dataframe

signal_proc = pd.read_csv(csv_path + '/multiple-disease-final-signal_processing.csv', index_col = 0)
y1 = signal_proc.disease1
y2 = signal_proc.disease2

zero_crossing = signal_proc.iloc[:,:21]
#zero_crossing #uncomment to see dataframe

tempogram = pd.read_csv(csv_path + '/p-value-selected-multiple-dis-temp.csv', index_col = 0)
#tempogram #uncomment to see dataframe

chromogram = pd.read_csv(csv_path + '/p-value-selected-multiple-dis-chromogram.csv', index_col = 0)
#chromogram #uncomment to see dataframe

w_lvl0 = pd.read_csv(csv_path + '/multiple-disease-wavelet-lvl0-coef.csv', index_col = 0).drop(['disease1', 'disease2'], axis =1)
w_lvl0 = w_lvl0.reset_index(drop=True)
#w_lvl0  #uncomment to see dataframe

This line will be used to encode the target value. It is not going to be used for other purposes.

In [4]:
signal_proc_t = pd.read_csv(csv_path + '/signal_processing.csv', index_col = 0)
labels= signal_proc_t.disease

In [5]:
features = pd.concat([spec, w_lvl0], axis = 1)
features = pd.concat([features, zero_crossing], axis = 1)
features = pd.concat([features, chromogram], axis = 1)
features = pd.concat([features, tempogram], axis = 1)

features

,freq_y1772,freq_z1773,freq_z1774,freq_x555,freq_y555,freq_z555,freq_x556,freq_y556,freq_z556,freq_x557,...,tempogram_y256,tempogram_z320,tempogram_y320,tempogram_x320,tempogram_z319,tempogram_y318,tempogram_z318,tempogram_x319,tempogram_y319,tempogram_z255
0,0.003383,0.002980,0.002855,0.002161,0.002094,0.002025,0.001971,0.001921,0.001869,0.001812,...,0.445113,1.978267e-09,2.062070e-08,9.183201e-08,2.766100e-07,0.000004,2.549855e-06,1.371246e-06,6.635903e-07,0.618049
1,0.004910,0.004310,0.004276,0.001875,0.002008,0.002132,0.002238,0.002317,0.002364,0.002378,...,0.538619,2.086170e-09,2.166218e-08,9.600761e-08,2.882793e-07,0.000005,2.655965e-06,1.426979e-06,6.905931e-07,0.654835
2,0.004735,0.004130,0.003788,0.002241,0.002232,0.002200,0.002142,0.002063,0.001966,0.001855,...,0.608095,2.098638e-09,2.191610e-08,9.778785e-08,2.947368e-07,0.000005,2.711798e-06,1.459085e-06,7.067081e-07,0.741303
3,0.004367,0.004345,0.004337,0.002082,0.002036,0.001993,0.001960,0.001941,0.001943,0.001987,...,0.715791,2.025586e-09,2.105369e-08,9.338860e-08,2.804590e-07,0.000004,2.585357e-06,1.388093e-06,6.717521e-07,0.788690
4,0.004148,0.003581,0.003203,0.001680,0.001666,0.001645,0.001618,0.001595,0.001595,0.001607,...,0.767121,1.932686e-09,2.006646e-08,8.889795e-08,2.668415e-07,0.000004,2.468796e-06,1.322954e-06,6.394051e-07,0.819613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,0.001908,0.002203,0.002238,0.002802,0.002739,0.002655,0.002559,0.002466,0.002393,0.002345,...,0.364195,2.059336e-09,2.124141e-08,9.317839e-08,2.766343e-07,0.000004,2.460065e-06,1.337205e-06,6.549272e-07,0.521096
323,0.001781,0.002088,0.002148,0.004011,0.003909,0.003803,0.003701,0.003584,0.003453,0.003323,...,0.277581,1.249031e-09,1.333401e-08,6.137947e-08,1.903869e-07,0.000003,1.851025e-06,9.828763e-07,4.674462e-07,0.671185
324,0.001544,0.001566,0.001578,0.002893,0.002796,0.002694,0.002628,0.002590,0.002553,0.002508,...,0.407483,1.720723e-09,1.811135e-08,8.183620e-08,2.500919e-07,0.000004,2.378340e-06,1.268592e-06,6.076013e-07,0.655393
325,0.000873,0.001025,0.001021,0.002950,0.002898,0.002809,0.002688,0.002544,0.002399,0.002275,...,0.270778,1.720426e-09,1.781081e-08,7.853572e-08,2.342196e-07,0.000004,2.101861e-06,1.139776e-06,5.565471e-07,0.476264


In [6]:
y = signal_proc.iloc[:,-2:].to_numpy()

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
one_hot = MultiLabelBinarizer()

print(one_hot.fit_transform(y))

print(one_hot.classes_)

[[0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 ...
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]]
['AR' 'AS' 'MR' 'MS']


In [8]:
y = one_hot.fit_transform(y)

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(features, y, train_size=0.8, random_state=88)

In [10]:
import xgboost as xgb
classifier = MultiOutputClassifier(xgb.XGBClassifier(objective = 'binary:logistic'))

clf = Pipeline([('classify', classifier)])


In [11]:
clf.fit(xtrain, ytrain)


Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               gamma=None,
                                                               gpu_id=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None, ...)))])

In [12]:
yhat = clf.predict(xtest)

In [13]:
df_results_test = pd.DataFrame({'AR': ytest[:,0], 'AS': ytest[:,1], 'MR': ytest[:,2], 'MS': ytest[:,3],
                        'predicted AR': yhat[:,0],
                          'predicted AS': yhat[:,1],
                          'predicted MR': yhat[:,2], 
                          'predicted MS': yhat[:,3]})
df_results_test

,AR,AS,MR,MS,predicted AR,predicted AS,predicted MR,predicted MS
0,1,1,0,0,1,1,0,0
1,1,1,0,0,1,1,1,0
2,1,0,1,0,0,1,1,0
3,0,1,1,0,1,0,1,0
4,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...
61,0,1,1,0,1,0,1,0
62,1,0,1,0,0,1,1,0
63,1,0,1,0,0,1,1,0
64,0,1,1,0,0,0,1,0


In [14]:
auc_y1 = roc_auc_score(ytest[:,0],yhat[:,0])
auc_y2 = roc_auc_score(ytest[:,1],yhat[:,1])
auc_y3 = roc_auc_score(ytest[:,2],yhat[:,2])
auc_y4 = roc_auc_score(ytest[:,3],yhat[:,3])

print("ROC AUC AR: %.4f, AS: %.4f, MR: %.4f, MS: %.4f " % (auc_y1, auc_y2, auc_y3, auc_y4))


ROC AUC AR: 0.4432, AS: 0.4761, MR: 0.9750, MS: 0.9783 


In [15]:
cm_y1 = confusion_matrix(ytest[:,0],yhat[:,0])
cm_y2 = confusion_matrix(ytest[:,1],yhat[:,1])
cm_y3 = confusion_matrix(ytest[:,2],yhat[:,2])
cm_y4 = confusion_matrix(ytest[:,3],yhat[:,3])
print(cm_y1)



[[25 19]
 [15  7]]


In [16]:
cm_y2

array([[11, 14],
       [20, 21]])

In [17]:
cm_y3

array([[19,  1],
       [ 0, 46]])

In [18]:
cm_y4

array([[43,  0],
       [ 1, 22]])

In [19]:
cr_y1 = classification_report(ytest[:,0],yhat[:,0])
cr_y2 = classification_report(ytest[:,1],yhat[:,1])
cr_y3 = classification_report(ytest[:,2],yhat[:,2])
cr_y4 = classification_report(ytest[:,3],yhat[:,3])

print (cr_y1)
print (cr_y2)
print (cr_y3)
print (cr_y4)




              precision    recall  f1-score   support

           0       0.62      0.57      0.60        44
           1       0.27      0.32      0.29        22

    accuracy                           0.48        66
   macro avg       0.45      0.44      0.44        66
weighted avg       0.51      0.48      0.49        66

              precision    recall  f1-score   support

           0       0.35      0.44      0.39        25
           1       0.60      0.51      0.55        41

    accuracy                           0.48        66
   macro avg       0.48      0.48      0.47        66
weighted avg       0.51      0.48      0.49        66

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       0.98      1.00      0.99        46

    accuracy                           0.98        66
   macro avg       0.99      0.97      0.98        66
weighted avg       0.99      0.98      0.98        66

              preci